In [4]:
# Libraries Loading
import pickle
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from keras.models import load_model
from matplotlib.ticker import MaxNLocator
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.metrics import (average_precision_score, confusion_matrix,
                             f1_score, get_scorer_names, log_loss, make_scorer,
                             precision_score, recall_score, roc_auc_score)
from sklearn.model_selection import (GridSearchCV, KFold, ShuffleSplit,
                                     cross_val_score, train_test_split)

2023-03-08 17:56:46.364246: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-08 17:56:46.490136: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-08 17:56:46.490168: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-08 17:56:47.498520: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
url = 'https://drive.google.com/file/d/1E9bFoMAwMVPMkZHmMQfWvSgaWoCLtKp4/view?usp=share_link'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
y_test_autoencoder = pd.read_pickle(url)

url = 'https://drive.google.com/file/d/1dknbQLJhQsbSFDNIpwjxZQwPc3fF14yF/view?usp=share_link'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
y_train_autoencoder = pd.read_pickle(url)


url = 'https://drive.google.com/file/d/105vmXogIQU8srmtdzfCHYQ9G-T7Iu5hl/view?usp=share_link'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
X_test_autoencoder = pd.read_pickle(url)

url = 'https://drive.google.com/file/d/1QemU274E0c14R8uknuipP20rNgDSLzcg/view?usp=share_link'
url = 'https://drive.google.com/uc?id=' + url.split('/')[-2]
X_train_autoencoder = pd.read_pickle(url)

print('1')

1


In [3]:
print(X_train_autoencoder.shape)
print(X_test_autoencoder.shape)
print(y_train_autoencoder.shape)
print(y_test_autoencoder.shape)

(796896, 10)
(103495, 10)
(796896,)
(103495,)


In [7]:
X_train = X_train_autoencoder
y_train = y_train_autoencoder
clf = IsolationForest(n_estimators=50, max_samples =0.1,contamination=0.01)
y_pred = clf.fit_predict(X_train)
y_pred_binary = [1 if p == -1 else 0 for p in y_pred]
y_pred_binary = np.array(y_pred_binary)
auc = roc_auc_score(y_train, y_pred_binary)
print('AUC: %.3f' % auc)

AUC: 0.493


In [15]:
param_grid = {'n_estimators': [50,100, 150],
              'max_samples': [0.1, 0.5, 0.9],
              'contamination': [0.01, 0.05, 0.1]}
best_params = {}
best_auc = 0

# split X_train using cv
cv = ShuffleSplit(n_splits=3, test_size=0.33, random_state=42) # 3-fold cv with 3 repeats``
for train_index, test_index in cv.split(X_train):
    # using same split for each hyperparameter
    X_train_cv, X_test_cv = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_cv, y_test_cv = y_train.iloc[train_index], y_train.iloc[test_index]

    # grid search
    for n_estimators in param_grid['n_estimators']:
        for max_samples in param_grid['max_samples']:
            for contamination in param_grid['contamination']:
                # fit model
                clf = IsolationForest(n_estimators=n_estimators,
                                max_samples=max_samples,
                                contamination=contamination)
                y_pred = clf.fit(X_train_cv)
                y_pred =y_pred.predict(X_train_cv)
                y_pred_binary = [1 if p == -1 else 0 for p in y_pred]
                y_pred_binary = np.array(y_pred_binary)
                auc = roc_auc_score(y_train_cv, y_pred_binary)
                print('AUC: %.3f' % auc)

                # update best params
                if auc > best_auc:
                    best_params['n_estimators'] = n_estimators
                    best_params['contamination'] = contamination
                    best_params['max_samples'] = max_samples
                    best_auc = auc

AUC: 0.493
AUC: 0.468
AUC: 0.440
AUC: 0.493
AUC: 0.470
AUC: 0.451
AUC: 0.493
AUC: 0.469
AUC: 0.456
AUC: 0.493
AUC: 0.466
AUC: 0.442
AUC: 0.494
AUC: 0.469
AUC: 0.448
AUC: 0.494
AUC: 0.469
AUC: 0.452
AUC: 0.493
AUC: 0.466
AUC: 0.440
AUC: 0.493
AUC: 0.470
AUC: 0.448
AUC: 0.493
AUC: 0.471
AUC: 0.449
AUC: 0.492
AUC: 0.465
AUC: 0.441
AUC: 0.493
AUC: 0.471
AUC: 0.455
AUC: 0.493
AUC: 0.473
AUC: 0.452
AUC: 0.492
AUC: 0.464
AUC: 0.446
AUC: 0.493
AUC: 0.470
AUC: 0.448
AUC: 0.493
AUC: 0.470
AUC: 0.449
AUC: 0.492
AUC: 0.468
AUC: 0.444
AUC: 0.493
AUC: 0.469
AUC: 0.447
AUC: 0.493
AUC: 0.471
AUC: 0.447
AUC: 0.493
AUC: 0.465
AUC: 0.445
AUC: 0.493
AUC: 0.470
AUC: 0.455
AUC: 0.493
AUC: 0.470
AUC: 0.450
AUC: 0.492
AUC: 0.466
AUC: 0.444
AUC: 0.493
AUC: 0.472
AUC: 0.447
AUC: 0.493
AUC: 0.469
AUC: 0.454
AUC: 0.492
AUC: 0.467
AUC: 0.444
AUC: 0.493
AUC: 0.469
AUC: 0.448
AUC: 0.493
AUC: 0.471
AUC: 0.448


Start from the best parameters

In [37]:
# keep a note to start from here
best_params = {'n_estimators': 50, 'max_samples': 0.9, 'contamination': 0.01}

In [38]:
# train the Isolation Forest model on the full training set with the best parameters
isoforest = IsolationForest(n_estimators=best_params['n_estimators'],
                            max_samples=best_params['max_samples'],
                            contamination=best_params['contamination'])
                            
isoforest.fit(X_train_autoencoder)

IsolationForest(contamination=0.01, max_samples=0.9, n_estimators=50)

In [43]:
# evaluate the model on the test set
y_pred = isoforest.predict(X_test_autoencoder)
y_pred_binary = [1 if p == -1 else 0 for p in y_pred]
roc_auc = roc_auc_score(y_test_autoencoder, y_pred_binary)
print('Test ROC AUC score:', roc_auc)

Test ROC AUC score: 0.5082760791734389


In [49]:
y_predicted = y_pred_binary
y_test = y_test_autoencoder
cm=confusion_matrix(y_test, y_predicted)
TN, FP, FN, TP = cm.ravel()

In [50]:
# false positive rate
FPR = FP/(FP+TN)
# false negative rate
FNR = FN/(FN+TP)
print(f'Test FPR: {FPR}')
print(f'Test FNR: {FNR}')

Test FPR: 0.017708532018913193
Test FNR: 0.9657393096342092


In [23]:
#convert to pickle file for latter running
pickle.dump(isoforest, open('IsolationForest.pkl', 'wb'))

In [25]:
# Load the model from the pickle file
with open('IsolationForest.pkl', 'rb') as f:
    model = pickle.load(f)

In [26]:
model

IsolationForest(contamination=0.01, max_samples=0.9)